In [ ]:
# imports 
import torch
def format_pytorch_version(version):
    return version.split('+')[0]
def format_cuda_version(version):
    return 'cu' + version.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install dgl

from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
import torch_geometric.utils as U
import torch.nn.functional as F
from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv

from torch import nn
from torch.optim import Adam
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import time

import dgl.nn as dglnn
import dgl
from dgl import AddSelfLoop
from dgl.data import CiteseerGraphDataset, CoraGraphDataset, PubmedGraphDataset
import pdb
from dgl.nn.pytorch import GATConv

In [12]:
#constants
DROPOUT_RATE=0.0
N_EPOCHS=50
REPEATS=100
HIDDEN_UNITS=8
HEADS=[8,1]

In [14]:
class lambda_value():
  def __init__(self,lambda_p):
    super().__init__()
    self.lambda_p=lambda_p

  def update_lambda(self,l):
    self.lambda_p=l
  
  def get_lambda(self):
    return self.lambda_p

latest_lambda = lambda_value(1.01)
  
  
class GATV2(nn.Module):
    def __init__(self, in_size, hid_size, out_size, heads, num_layers=2):
        super().__init__()
        self.gat_layers = nn.ModuleList()
        
        self.gat_layers.append(
            dglnn.GATv2Conv(
                in_size,
                hid_size,
                heads[0],
                feat_drop=DROPOUT_RATE,
                attn_drop=DROPOUT_RATE,
                activation=F.elu,
            )
        )

        #middle layers
        for n in range(num_layers-2):
           self.gat_layers.append(
            dglnn.GATv2Conv(
                hid_size * heads[0],
                hid_size,
                heads[0],
                feat_drop=DROPOUT_RATE,
                attn_drop=DROPOUT_RATE,
                activation=F.elu,
            )
        )

        self.gat_layers.append(
            dglnn.GATv2Conv(
                hid_size * heads[0],
                out_size,
                heads[1],
                feat_drop=DROPOUT_RATE,
                attn_drop=DROPOUT_RATE,
                activation=None,
            )
        )



    def forward(self, g, inputs):
        h = inputs
        for i, layer in enumerate(self.gat_layers):
            h = layer(g, h)
            if i == len(self.gat_layers)-1:  
                h = h.mean(1)
            else: 
                h = h.flatten(1)
        return h



class GATReg(nn.Module):
    def __init__(self, in_size, hid_size, out_size, heads, lambda_parameter=1.01, num_layers=2):
        super().__init__()
        self.gat_layers = nn.ModuleList()
        self.lambda_parameter=lambda_parameter

  
        self.gat_layers.append(
            dglnn.GATv2Conv(
                in_size,
                hid_size,
                heads[0],
                feat_drop=DROPOUT_RATE,
                attn_drop=DROPOUT_RATE,
                activation=F.elu,
            )
        )
        #middle layers
        for n in range(num_layers-2):
           self.gat_layers.append(
            dglnn.GATv2Conv(
                hid_size * heads[0],
                hid_size,
                heads[0],
                feat_drop=DROPOUT_RATE,
                attn_drop=DROPOUT_RATE,
                activation=F.elu,
            )
        )
           
        self.gat_layers.append(
            dglnn.GATv2Conv(
                hid_size * heads[0],
                out_size,
                heads[1],
                feat_drop=DROPOUT_RATE,
                attn_drop=DROPOUT_RATE,
                activation=None,
            )
        )

    def forward(self, g, inputs):
        h = inputs
        beta = torch.distributions.beta.Beta(torch.tensor([self.lambda_parameter], device="cpu"), torch.tensor([self.lambda_parameter], device="cpu")) 
        scalar = beta.sample(h.shape[:1]) + 0.1
        scalar[scalar > 1] = 1.0/(2.0-scalar[scalar > 1])

        for i, layer in enumerate(self.gat_layers):
            h = layer(g, h)
            if i == len(self.gat_layers)-1:  
                h = h.mean(1)

            else:  
                h = h.flatten(1)
        h = scalar * h
        
        return h


class backward_reg(torch.autograd.Function):

    @staticmethod
    def forward(xx, x, training):
        if training:
            lambda_p = latest_lambda.get_lambda()
            beta = torch.distributions.beta.Beta(torch.tensor([lambda_p], device="cpu"), torch.tensor([lambda_p], device="cpu"))
            lambdaa = beta.sample(x.shape[:1]) + 0.5
            lambdaa[lambdaa > 1] = 1./(2-lambdaa[lambdaa> 1])
            x = lambdaa * x
        
        return x

    @staticmethod
    def backward(xx, gradients):

        lambda_p = latest_lambda.get_lambda()
        beta = torch.distributions.beta.Beta(torch.tensor([lambda_p], device="cpu"), torch.tensor([lambda_p], device="cpu"))
        lambdaa = beta.sample(gradients.shape[:1]) + .5
        lambdaa[lambdaa > 1] = 1./(2-lambdaa[lambdaa > 1])

        return lambdaa * gradients, None


class GATReg2(nn.Module):
    def __init__(self, in_size, hid_size, out_size, heads, lambda_parameter=0.05, num_layers=2):
        super().__init__()
        self.gat_layers = nn.ModuleList()
        self.lambda_parameter=lambda_parameter
        
        # two-layer GAT
        self.gat_layers.append(
            dglnn.GATv2Conv(
                in_size,
                hid_size,
                heads[0],
                feat_drop=DROPOUT_RATE,
                attn_drop=DROPOUT_RATE,
                activation=F.elu,
            )
        )

        #middle layers
        for n in range(num_layers-2):
           self.gat_layers.append(
            dglnn.GATv2Conv(
                hid_size * heads[0],
                hid_size,
                heads[0],
                feat_drop=DROPOUT_RATE,
                attn_drop=DROPOUT_RATE,
                activation=F.elu,
            )
        )
           
        self.gat_layers.append(
            dglnn.GATv2Conv(
                hid_size * heads[0],
                out_size,
                heads[1],
                feat_drop=DROPOUT_RATE,
                attn_drop=DROPOUT_RATE,
                activation=None,
            )
        )

    def forward(self, g, inputs):
        h = inputs
        beta = torch.distributions.beta.Beta(torch.tensor([self.lambda_parameter], device="cpu"), torch.tensor([self.lambda_parameter], device="cpu")) 
        scalar = beta.sample(h.shape[:1]) + 0.5
        scalar[scalar > 1.0] = 1.0/(2.0-scalar[scalar > 1])
        for i, layer in enumerate(self.gat_layers):
            h = layer(g, h)
            if i == len(self.gat_layers)-1: 
                h = h.mean(1)
            else: 
                h = h.flatten(1)
        #h=h*scalar
        return h

def evaluate(g, features, labels, mask, model):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


def train(g, features, labels, masks, model):
    # define train/val samples, loss function and optimizer
    train_mask = masks[0]
    val_mask = masks[1]
    loss_fcn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=5e-4)

    for epoch in range(N_EPOCHS):
        model.train()
        bw_reg=backward_reg(model.lambda_parameter)
        logits = model(g, features)
        beta = torch.distributions.beta.Beta(torch.tensor([model.lambda_parameter], device="cpu"), torch.tensor([model.lambda_parameter], device="cpu")) 
        scalar = beta.sample(logits.shape[:1]) + .1
        scalar[scalar > 1] = 1./(2.0-scalar[scalar > 1])
        loss = loss_fcn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        bw_reg.apply(logits[train_mask], model.training)
        optimizer.step()
        acc = evaluate(g, features, labels, val_mask, model)

In [ ]:
!pip install dgl
import dgl

In [ ]:
%%time
accuracies={}
noise=[i/100 for i in range(0,80)]
for noise_level in noise:
    nl_acc_avg=0
    nl_accs=[]
  # transform dataset
    transform = (dgl.AddEdge(noise_level))    
    data=PubmedGraphDataset(transform=transform)
    g = data[0]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    g = g.int().to(device)
    features = g.ndata["feat"]
    labels = g.ndata["label"]
    masks = g.ndata["train_mask"], g.ndata["val_mask"], g.ndata["test_mask"]
    in_size = features.shape[1]
    out_size = data.num_classes

    for i in range(REPEATS):
      model = GATV2(in_size, HIDDEN_UNITS, out_size, heads=[8, 1], num_layers=2).to(device)

    # model training
      train(g, features, labels, masks, model)
      acc = evaluate(g, features, labels, masks[2], model)
      nl_accs.append(acc)
      
    print("Noise level: ", noise_level)
    mean_acc=np.mean(np.asarray(nl_accs))
    accuracies[noise_level]=mean_acc
    std_acc=np.std(np.asarray(nl_accs))
    print(f'\nMean test accuracy: {mean_acc}%\n')
    print(f'\nStd test accuracy: {std_acc}%\n')
        

In [ ]:
%%time
accuracies={}
noise=[i/100 for i in range(0,80)]
for noise_level in noise:
    nl_acc_avg=0
    nl_accs=[]
  # transform dataset
    transform = (dgl.AddEdge(noise_level))    
    data=PubmedGraphDataset(transform=transform)
    g = data[0]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    g = g.int().to(device)
    features = g.ndata["feat"]
    labels = g.ndata["label"]
    masks = g.ndata["train_mask"], g.ndata["val_mask"], g.ndata["test_mask"]
    in_size = features.shape[1]
    out_size = data.num_classes
    
    for i in range(REPEATS):
      model = GATReg(in_size, HIDDEN_UNITS, out_size, heads=[8, 1], num_layers=2).to(device)

    # model training
      train(g, features, labels, masks, model)
      acc = evaluate(g, features, labels, masks[2], model)
      nl_accs.append(acc)
      
    print("Noise level: ", noise_level)
    mean_acc=np.mean(np.asarray(nl_accs))
    accuracies[noise_level]=mean_acc
    std_acc=np.std(np.asarray(nl_accs))
    print(f'\nMean test accuracy: {mean_acc}%\n')
    print(f'\nStd test accuracy: {std_acc}%\n')
        

In [ ]:
%%time
accuracies={}
noise=[i/100 for i in range(0,80)]
for noise_level in noise:
    nl_acc_avg=0
    nl_accs=[]
  # transform dataset
    transform = (dgl.AddEdge(noise_level))    
    data=PubmedGraphDataset(transform=transform)
    g = data[0]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    g = g.int().to(device)
    features = g.ndata["feat"]
    labels = g.ndata["label"]
    masks = g.ndata["train_mask"], g.ndata["val_mask"], g.ndata["test_mask"]
    in_size = features.shape[1]
    out_size = data.num_classes
    
    for i in range(REPEATS):
      model = GATReg2(in_size, HIDDEN_UNITS, out_size, heads=[8, 1], num_layers=2).to(device)

    # model training
      train(g, features, labels, masks, model)
      acc = evaluate(g, features, labels, masks[2], model)
      nl_accs.append(acc)
      
    print("Noise level: ", noise_level)
    mean_acc=np.mean(np.asarray(nl_accs))
    accuracies[noise_level]=mean_acc
    std_acc=np.std(np.asarray(nl_accs))
    print(f'\nMean test accuracy: {mean_acc}%\n')
    print(f'\nStd test accuracy: {std_acc}%\n')

In [ ]:
%%time
# lambda 0.005
accuracies={}
noise=[0.0,0.01,0.02,0.05,0.08,0.1,0.12,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.6]
for noise_level in noise:
    nl_acc_avg=0
    nl_accs=[]
  # transform dataset
    transform = (dgl.AddEdge(noise_level))    
    data = CoraGraphDataset(transform=transform)
    g = data[0]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    g = g.int().to(device)
    features = g.ndata["feat"]
    labels = g.ndata["label"]
    masks = g.ndata["train_mask"], g.ndata["val_mask"], g.ndata["test_mask"]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    g = g.int().to(device)
    features = g.ndata["feat"]
    labels = g.ndata["label"]

    masks = g.ndata["train_mask"], g.ndata["val_mask"], g.ndata["test_mask"]

    in_size = features.shape[1]
    out_size = data.num_classes
    for i in range(REPEATS):
      model = GATReg(in_size, HIDDEN_UNITS, out_size, heads=[8, 1], num_layers=2).to(device)

    # model training
      train(g, features, labels, masks, model)
      acc = evaluate(g, features, labels, masks[2], model)
      nl_accs.append(acc)
      
    print("Noise level: ", noise_level)
    mean_acc=np.mean(np.asarray(nl_accs))
    accuracies[noise_level]=mean_acc
    std_acc=np.std(np.asarray(nl_accs))
    print(f'\nMean test accuracy: {mean_acc}%\n')
    print(f'\nStd test accuracy: {std_acc}%\n')
        